# モデルを解釈する

Azure Machine Learning を使用して、各機能が予測ラベルに与える影響の量を定量化する *Explainer* を使用して、モデルを解釈できます。一般的な Explainer は多く、それぞれ異なる種類のモデリング アルゴリズムに適しています。ただし、それらを使用する基本的なアプローチは同じです。

## SDK パッケージのインストール

このノートブックのコードを実行するには、最新バージョンの **azureml-sdk** および **azureml-widgets** パッケージに加えて、**azureml-explain-model** パッケージが必要です。また、Azure ML 解釈可能性ライブラリ (**azureml-interpret**) も使用します。これを使用すると、Azure ML 実験でトレーニングされていない場合や、Azure ML ワークスペースに登録されていない場合でも、多くの一般的な種類のモデルを解釈できます。

次のセルを実行して、これらのパッケージがインストールされていることを確認します。 

In [ ]:
!pip show azureml-explain-model azureml-interpret

## モデルを説明する

Azure Machine Learning の外部でトレーニングされたモデルから始めましょう - 下のセルを実行して、デシジョン ツリー分類モデルをトレーニングします。

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 糖尿病データセットを読み込む
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# 特徴とラベルを分離する
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

トレーニング プロセスでは、ホールドバック検証データセットに基づいてモデル評価メトリックが生成されるため、予測の精度を把握できます。しかし、データの特徴は予測にどのような影響を与えるのでしょうか?

### モデルの説明を取得する

先にインストールした Azure ML の解釈可能性ライブラリから、モデルに適した Explainer を取得しましょう。Explainer には多くの種類があります。この例では、適切な [SHAP](https://github.com/slundberg/shap) モデル Explainer を呼び出すことによって、多くの種類のモデルを説明するために使用できる「ブラック ボックス」の説明である表形式の *Explainer* を使用します。

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# 「特徴」と「クラス」フィールドはオプションです
tab_explainer = TabularExplainer(model,
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### *グローバル*な特徴の重要度を取得する

最初に行うことは、全体的な*特徴の重要度*を評価することによってモデルを説明しようとすることです - つまり、各特徴がトレーニング データセット全体に基づいて予測に影響を与える程度を定量化します。

In [ ]:
# ここでトレーニング データまたはテスト データを使用できます
global_tab_explanation = tab_explainer.explain_global(X_train)

# 重要度別の上位の特徴を取得する
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

特徴の重要度が順位付けされ、最も重要な機能が最初に表示されます。

### *ローカル*な特徴の重要度を取得する

全体的な見解がありますが、個々の観察を説明はどうですか? 可能性のある各ラベル値を予測する決定に各機能が影響を与えた程度を定量化して、個々の予測の*ローカル*説明を生成しましょう。この場合、バイナリ モデルであるため、2 つのラベル (糖尿病以外と糖尿病) があります。また、データセット内の個々の観測値に対するこれらのラベル値の各特徴の影響を定量化できます。テスト データセットの最初の 2 つのケースを評価するだけです。

In [ ]:
# 説明したい観測値を取得する (最初の 2 つ)
X_explain = X_test[0:2]

# 予測を取得する
predictions = model.predict(X_explain)

# ローカルな説明を取得する
local_tab_explanation = tab_explainer.explain_local(X_explain)

# 各ラベルの特徴の名前と重要度を取得する
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])

## モデル トレーニング実験に説明可能性を追加する

これまで見てきたように、Azure Machine Learning の外部でトレーニングされたモデルの説明を生成できます。ただし、Azure Machine Learning ワークスペースでモデルをトレーニングして登録するために実験を使用する場合は、モデルの説明を生成してログに記録できます。

次のセルでコードを実行して、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 実験を使用してモデルをトレーニングして説明する

では、実験を作成して、必要なファイルをローカル フォルダーに配置しましょう - この場合、糖尿病データの同じ CSV ファイルを使用してモデルをトレーニングします。

In [ ]:
import os, shutil
from azureml.core import Experiment

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

次の特徴を含む以外 Azure ML トレーニング スクリプトと同様のトレーニング スクリプトを作成します。

- 以前使用したモデルの説明を生成する同じライブラリがインポートされ、グローバルな説明を生成するために使用されます
- **ExplanationClient** ライブラリを使用して、説明を実験出力にアップロードします

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Azure ML 実行ライブラリをインポートする
from azureml.core.run import Run

# モデルの説明用ライブラリをインポートする
from azureml.interpret import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# 特徴とラベルを分離する
X, y = data[features].values, data['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# 説明を取得する
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Explanation Client を取得し、説明をアップロードする
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# 実行を完了する
run.complete()

これで実験を実行できます。トレーニング環境には **azureml-interpret** ライブラリが含まれているので、スクリプトは **TabularExplainer** を作成して **ExplainerClient** クラスを使用できる点に留意してください。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 実験用 Python 環境を作成する
explain_env = Environment("explain-env")

# パッケージの依存関係のセットを作成する (azureml-interpret パッケージを含む)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
explain_env.python.conda_dependencies = packages

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder,
                      script='diabetes_training.py',
                      environment=explain_env) 

# 実験を送信する
experiment_name = 'mslearn-diabetes-explain'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

## 特徴の重要度の値を取得する

実験の実行が完了したら、**ExplanationClient** クラスを使用して、実行用に登録された説明から特徴の重要度を取得できます。

In [ ]:
from azureml.interpret import ExplanationClient

# 特徴の説明を取得する
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# 全体的な特徴の重要度
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studio でモデルの説明を表示する

また、実行の詳細ウィジェットの**実行の詳細を表示**リンクをクリックすると、Azure Machine Learning Studio の実行が表示され、**説明**タブを表示できます。次に以下を実行します。

1. 表形式の Explainer の説明 ID を選択します。
2. 全体的なグローバル特徴の重要度を示す**特徴の重要度の集計**グラフを表示します。
3. テスト データの各データ ポイントを示す**個別の特徴の重要度**グラフを表示します。
4. 個々のポイントを選択すると、選択したデータ ポイントの個々の予測のローカル特徴の重要度が表示されます。


**詳細情報**: Azure ML での Explainer の使用の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability)を参照してください。 